# Create Batch Job for OpenAI/Azure API

This notebook creates a batch job for processing multiple prompts through OpenAI or Azure OpenAI API.

## Features
- Supports GPT-4.1, o3, and GPT-5 models
- Optional Azure OpenAI deployment
- Reasoning effort configuration for o3/GPT-5 (low, medium, high)
- Batch metadata saved for result retrieval

## Prerequisites
- Set the `OPENAI_API_KEY` environment variable (or `AZURE_OPENAI_API_KEY` for Azure)
- Prepare input JSON file with `keys` and `contexts` arrays

## Input Format
```json
{
  "keys": ["case_001", "case_002", ...],
  "contexts": [
    [{"role": "user", "content": "..."}],
    [{"role": "system", "content": "..."}, {"role": "user", "content": "..."}],
    ...
  ]
}
```

## Reasoning Effort (o3/GPT-5 only)
- `low`: Faster, less thorough reasoning
- `medium`: Balanced (default for o3/GPT-5)
- `high`: Most thorough reasoning, slower

## Output
- JSONL file uploaded to OpenAI
- Batch metadata saved to `logs/{INPUT_DIR}/{MODEL_NAME}/{INPUT_FILE}_thinking_{effort}.json`
- Use `retrieve_batch_results_gpt.ipynb` to get results after batch completes

In [ ]:
# ============================================================================
# CONFIGURATION - Edit these variables before running
# ============================================================================
from pathlib import Path

# Base directory for data files
BASE_DIR = Path(".")  # Change to your data directory

# Input file settings
INPUT_DIR = "data"  # Directory containing input JSON file
INPUT_FILE = "your_input_file"  # Name without .json extension

# Model settings
# Supported models: gpt-4.1-2025-04-14, o3-2025-04-16, gpt-5-2025-08-07
MODEL_NAME = "gpt-4.1-2025-04-14"

# Reasoning effort (required for o3 and gpt-5, ignored for other models)
# Options: "low", "medium", "high", or None
REASONING_EFFORT = "medium"

# Azure OpenAI settings (set USE_AZURE=True to use Azure)
USE_AZURE = False
AZURE_ENDPOINT = "https://your-resource.openai.azure.com/"  # Your Azure endpoint
AZURE_DEPLOYMENT_MAP = {  # Map model names to Azure deployment names
    "o3-2025-04-16": "o3-batch",
    "gpt-4.1-2025-04-14": "gpt-4.1-batch",
}

# Output directories
LOGS_DIR = BASE_DIR / "logs"
JSONL_DIR = BASE_DIR / "openai_jsonl"

# ============================================================================

In [ ]:
import json
import os
from openai import OpenAI, AzureOpenAI

# Determine if reasoning effort applies to this model
is_reasoning_model = "o3" in MODEL_NAME or "gpt-5" in MODEL_NAME
thinking_effort = REASONING_EFFORT if is_reasoning_model else None

# Default to medium for reasoning models if not specified
if is_reasoning_model and not thinking_effort:
    thinking_effort = "medium"

# Get Azure deployment name if using Azure
deployment_name = None
if USE_AZURE:
    if MODEL_NAME not in AZURE_DEPLOYMENT_MAP:
        raise ValueError(f"Model {MODEL_NAME} not found in AZURE_DEPLOYMENT_MAP")
    deployment_name = AZURE_DEPLOYMENT_MAP[MODEL_NAME]

# Initialize client
if USE_AZURE:
    client = AzureOpenAI(
        api_version="2025-03-01-preview",
        azure_endpoint=AZURE_ENDPOINT,
        api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    )
else:
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

# Build paths
input_path = BASE_DIR / INPUT_DIR / f"{INPUT_FILE}.json"
thinking_suffix = f"_thinking_{thinking_effort}" if thinking_effort else ""
jsonl_path = JSONL_DIR / INPUT_DIR / MODEL_NAME / f"{INPUT_FILE}{thinking_suffix}.jsonl"
jsonl_path.parent.mkdir(parents=True, exist_ok=True)

# Load input data
with open(input_path, 'r') as f:
    data = json.load(f)

print(f"Input file: {input_path}")
print(f"Loaded {len(data['keys'])} cases")
print(f"Model: {MODEL_NAME}")
print(f"Using Azure: {USE_AZURE}")
if thinking_effort:
    print(f"Reasoning effort: {thinking_effort}")

In [ ]:
# Build JSONL batch requests
requests = []
key_dict = {}

for i, (key, context) in enumerate(list(zip(data["keys"], data["contexts"]))):
    batch_request = {
        "custom_id": str(i),
        "method": "POST",
        "url": "/chat/completions" if USE_AZURE else "/v1/chat/completions",
        "body": {
            "model": deployment_name if USE_AZURE else MODEL_NAME,
            "messages": context,
            "temperature": 1.0 if is_reasoning_model else 0.7,
            "top_p": 1.0,
            "max_completion_tokens": 50000 if is_reasoning_model else 32000,
        }
    }
    
    # Add reasoning_effort for o3/gpt-5 models
    if thinking_effort:
        batch_request["body"]["reasoning_effort"] = thinking_effort

    key_dict[str(i)] = key
    requests.append(batch_request)

# Write to JSONL file
with open(jsonl_path, "w") as f:
    for r in requests:
        json.dump(r, f)
        f.write("\n")

print(f"Created {len(requests)} batch requests")
print(f"Saved to: {jsonl_path}")

In [ ]:
# Upload JSONL file and create batch job
batch_input_file = client.files.create(
    file=open(jsonl_path, "rb"),
    purpose="batch"
)

print(f"Uploaded file:")
print(f"  ID: {batch_input_file.id}")
print(f"  Filename: {batch_input_file.filename}")
print(f"  Status: {batch_input_file.status}")

In [ ]:
# Create batch job
batch_result = client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/chat/completions" if USE_AZURE else "/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "batch processing",
    }
)

print(f"Created batch job:")
print(f"  ID: {batch_result.id}")
print(f"  Status: {batch_result.status}")

In [ ]:
# Check batch status (re-run this cell to check progress)
batch_result = client.batches.retrieve(batch_result.id)

print(f"Batch ID: {batch_result.id}")
print(f"Status: {batch_result.status}")
print(f"\nRequest counts:")
print(f"  Completed: {batch_result.request_counts.completed}")
print(f"  Failed: {batch_result.request_counts.failed}")
print(f"  Total: {batch_result.request_counts.total}")

if batch_result.status == "completed":
    print(f"\nBatch completed!")
    print(f"Output file ID: {batch_result.output_file_id}")
elif batch_result.status in ["validating", "in_progress", "finalizing"]:
    print(f"\nBatch still processing...")
elif batch_result.status == "failed":
    print(f"\nBatch failed!")
    if batch_result.errors:
        print(f"Errors: {batch_result.errors}")

In [ ]:
# Save batch metadata and key_dict for later result retrieval
message_batch = batch_result.to_dict()

# Build log path
log_save_path = LOGS_DIR / INPUT_DIR / MODEL_NAME / f"{INPUT_FILE}{thinking_suffix}.json"
log_save_path.parent.mkdir(parents=True, exist_ok=True)

# Save batch metadata and key mapping
saving_dict = {
    "message_batch": message_batch,
    "key_dict": key_dict
}

with open(log_save_path, 'w') as f:
    json.dump(saving_dict, f, indent=4, default=str)

print(f"Batch metadata saved to: {log_save_path}")
print(f"\nBatch ID: {batch_result.id}")
print(f"\nTo check status later, use:")
print(f"  client.batches.retrieve('{batch_result.id}')")
print(f"\nTo retrieve results when complete, use retrieve_batch_results_gpt.ipynb")